### Streamlined

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from segment_anything import sam_model_registry, SamPredictor

In [ ]:
image_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\images"
mask_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\masks"
visual_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\visualization"
review_image_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_images"
review_mask_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_masks"
review_visual_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_visualizations"

os.makedirs(mask_output_dir, exist_ok=True)
os.makedirs(visual_output_dir, exist_ok=True)
os.makedirs(review_image_dir, exist_ok=True)
os.makedirs(review_mask_dir, exist_ok=True)
os.makedirs(review_visual_output_dir, exist_ok=True)

In [ ]:
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)

image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(".jpg")])

MIN_MASK_AREA = 5000     
MIN_MEAN_Y = 500 

In [ ]:
for image_name in tqdm(image_files, desc="Generating masks"):
    image_path = os.path.join(image_dir, image_name)

    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    predictor.set_image(image_rgb)

    input_point = np.array([
        [300, 600], [500, 620], [700, 610], 
        [100, 200], [1000, 200] 
    ])
    input_label = np.array([1, 1, 1, 0, 0])

    input_box = np.array([[100, 550, 1100, 700]])

    masks, scores, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True
    )

    best_mask = masks[np.argmax(scores)].astype(np.uint8) * 255
    mask_filename = image_name.replace(".jpg", "_mask.png")
    overlay_filename = image_name.replace(".jpg", "_overlay.jpg")

    mask_area = cv2.countNonZero(best_mask)
    ys, xs = np.where(best_mask == 255)
    mean_y = np.mean(ys) if len(ys) > 0 else 0

    is_low_quality = (mask_area < MIN_MASK_AREA or mean_y < MIN_MEAN_Y)

    overlay = image_rgb.copy()
    overlay[best_mask == 255] = [255, 0, 0]  # Red overlay
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

    if is_low_quality:
        print(f"Flagged for review: {image_name} | Area: {mask_area} | Mean Y: {mean_y:.1f}")
        shutil.copy(image_path, os.path.join(review_image_dir, image_name))
        cv2.imwrite(os.path.join(review_mask_dir, mask_filename), best_mask)
        cv2.imwrite(os.path.join(review_visual_output_dir, overlay_filename), overlay_bgr)
    else:
        cv2.imwrite(os.path.join(mask_output_dir, mask_filename), best_mask)
        cv2.imwrite(os.path.join(visual_output_dir, overlay_filename), overlay_bgr)

print("Done. Masks generated, visualizations saved, and low-quality results flagged for review.")

### One Image

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from segment_anything import sam_model_registry, SamPredictor

image_name = "image_0101.jpg"
base_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\valid"

image_dir = os.path.join(base_dir, "images")
mask_output_dir = os.path.join(base_dir, "masks")
visual_output_dir = os.path.join(base_dir, "visualizations")

MIN_MASK_AREA = 5000
MIN_MEAN_Y = 500

checkpoint_path = r"C:\Users\ksaks\Documents\Masters Stuff\PR\sam_vit_b_01ec64.pth"
sam = sam_model_registry["vit_b"](checkpoint=checkpoint_path)
predictor = SamPredictor(sam)

image_path = os.path.join(image_dir, image_name)
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

predictor.set_image(image_rgb)

input_point = np.array([
    [300, 600], [500, 620], [700, 610], 
    [100, 200], [1000, 200]
])
input_label = np.array([1, 1, 1, 0, 0])

input_box = np.array([[100, 550, 1100, 700]])

masks, scores, _ = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True
)

best_mask = masks[np.argmax(scores)].astype(np.uint8) * 255
mask_filename = image_name.replace(".jpg", "_mask.png")
overlay_filename = image_name.replace(".jpg", "_overlay.jpg")

mask_area = cv2.countNonZero(best_mask)
ys, xs = np.where(best_mask == 255)
mean_y = np.mean(ys) if len(ys) > 0 else 0
is_low_quality = (mask_area < MIN_MASK_AREA or mean_y < MIN_MEAN_Y)

overlay = image_rgb.copy()
overlay[best_mask == 255] = [255, 0, 0]  # Red
overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)


cv2.imwrite(os.path.join(mask_output_dir, mask_filename), best_mask)
cv2.imwrite(os.path.join(visual_output_dir, overlay_filename), overlay_bgr)

plt.figure(figsize=(10, 5))
plt.imshow(image_rgb)
plt.imshow(best_mask, alpha=0.5, cmap='autumn')
plt.title("Predicted Ground Mask (Overlay)")
plt.axis('off')
plt.show()

### New SAM

In [ ]:
import os
import cv2
import numpy as np
import shutil
from tqdm import tqdm
from segment_anything import sam_model_registry, SamPredictor

image_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\images"
mask_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\masks"
visual_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\visualization"
review_image_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_images"
review_mask_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_masks"
review_visual_output_dir = r"C:\Users\ksaks\Documents\Masters Stuff\PR\EuroPallet - Bad\test\review_visualizations"

MIN_MASK_AREA = 5000
MIN_MEAN_Y = 300

os.makedirs(mask_output_dir, exist_ok=True)
os.makedirs(visual_output_dir, exist_ok=True)
os.makedirs(review_image_dir, exist_ok=True)
os.makedirs(review_mask_dir, exist_ok=True)
os.makedirs(review_visual_output_dir, exist_ok=True)

sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
predictor = SamPredictor(sam)

image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".png", ".jpeg"))]

for image_name in tqdm(image_files, desc="Generating masks"):
    image_path = os.path.join(image_dir, image_name)
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    predictor.set_image(image_rgb)

    h, w, _ = image_rgb.shape

    input_point = np.array([
        [int(w * 0.25), int(h * 0.85)],
        [int(w * 0.50), int(h * 0.90)],
        [int(w * 0.75), int(h * 0.85)],
        [int(w * 0.50), int(h * 0.50)],
        [int(w * 0.50), int(h * 0.30)]
    ])
    input_label = np.array([1, 1, 1, 0, 0])

    masks, scores, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True
    )

    best_mask = masks[np.argmax(scores)].astype(np.uint8) * 255

    mask_area = cv2.countNonZero(best_mask)
    ys, xs = np.where(best_mask == 255)
    mean_y = np.mean(ys) if len(ys) > 0 else 0
    is_low_quality = (mask_area < MIN_MASK_AREA or mean_y < MIN_MEAN_Y)

    overlay = image_rgb.copy()
    overlay[best_mask == 255] = [0, 255, 0]
    overlay_bgr = cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR)

    mask_filename = os.path.splitext(image_name)[0] + "_mask.png"
    overlay_filename = os.path.splitext(image_name)[0] + "_overlay.jpg"

    if is_low_quality:
        shutil.copy(image_path, os.path.join(review_image_dir, image_name))
        cv2.imwrite(os.path.join(review_mask_dir, mask_filename), best_mask)
        cv2.imwrite(os.path.join(review_visual_output_dir, overlay_filename), overlay_bgr)
    else:
        cv2.imwrite(os.path.join(mask_output_dir, mask_filename), best_mask)
        cv2.imwrite(os.path.join(visual_output_dir, overlay_filename), overlay_bgr)

print("All done! Masks and visualizations saved.")